In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-11-27 2021-11-30


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|███████████████▏                                                                               | 81/505 [01:00<04:33,  1.55it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|████████████████████████████████████████████████████████▉                                     | 306/505 [03:29<02:17,  1.45it/s]

project_market prices documents must be a non-empty list


 99%|█████████████████████████████████████████████████████████████████████████████████████████████▍| 502/505 [05:42<00:02,  1.40it/s]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████▊| 504/505 [05:43<00:00,  1.51it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [05:44<00:00,  1.47it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████▏                                                                                  | 65/505 [02:02<13:51,  1.89s/it]

'date'


 16%|██████████████▊                                                                                | 79/505 [02:29<13:26,  1.89s/it]

'date'


 99%|█████████████████████████████████████████████████████████████████████████████████████████████▍| 502/505 [16:05<00:05,  1.89s/it]

...

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████▊| 504/505 [16:09<00:01,  1.87s/it]

'date'


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [16:11<00:00,  1.92s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
315,2.298219,0.080389,0.092950,0.0,0.117950,MRNA,Moderna,Health Care
277,0.433408,0.194496,0.188177,0.0,0.060064,LRCX,Lam Research,Information Technology
345,1.547018,0.609102,0.292291,0.0,0.059455,NVDA,Nvidia,Information Technology
437,0.298857,0.410889,0.106296,0.0,0.055347,TER,Teradyne,Information Technology
45,0.750094,0.171068,0.083937,0.0,0.055342,AMAT,Applied Materials,Information Technology
410,0.775976,0.232792,0.132318,0.0,0.055161,STX,Seagate Technology,Information Technology
319,0.553939,0.168171,0.066203,0.0,0.054764,MPWR,Monolithic Power Systems,Information Technology
347,0.393769,0.160973,0.091698,0.0,0.054083,NXPI,NXP,Information Technology
311,-0.379713,-0.447318,0.097529,0.0,0.054078,MCHP,Microchip Technology,Information Technology
438,0.558011,0.466668,-0.059243,0.0,0.050900,TSLA,Tesla,Consumer Discretionary


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Information Technology,0.263914,0.068918,0.019969,0.000000,0.025341
Utilities,0.116987,0.043158,0.000583,0.000000,0.012953
Real Estate,0.446742,0.063258,0.007447,0.000000,0.009558
Energy,0.596798,0.024899,-0.061362,0.000391,0.008823
Materials,0.274527,0.064740,0.018120,0.000000,0.004481
Industrials,0.264083,0.047968,-0.009533,0.000000,0.004195
Consumer Discretionary,0.278470,0.051411,-0.011752,0.000000,0.003445
Financials,0.363200,0.019421,-0.033095,0.000000,0.003157
Health Care,0.178125,-0.009513,-0.031599,0.000000,0.002919
